# Supervised Classification Urban Change Detection
This notebook is the complete workflow for urban change detection algorithm.
The goal of this process is to be able to identigfy pixels that have been urbanised (changed from vegetation to the built environment) during the operation of the LandSat Earth Observation Satellites (since 1987).

This notebook lets you:
- create training data to train the classifier on
- classify the data according to 4 broad landcover classes
- view the results of your classification process
- identify if and when a pixel that was previously not urban becomes dominantly urban (change detection)
- view the results of the change detection

The markdown cells have been designed to work with the 'Table Of Contents(2)' Jupyter notebook extension.
This is highly recommended, if you don't have it yet (and are working on the VDI on the 'agdc-py3-prod module'
select "Edit" on the menu bar above, click the "nbextension config" button at the bottom of the menu, and enable
the extension. The 'Collapsible Headings' extension is also highly recommended.

This was written Mike Barnes as part of his third graduate rotation, during January 2018.
Any questions, please contact me at michael.barnes@ga.gov.au

## Python Library Imports

In [ ]:
%matplotlib notebook
import os

import numpy as np
import pandas as pd
import xarray as xr

import datacube
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict

from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.colors as colors
import matplotlib.patches as mpatches

import gdal

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, Dropdown
from IPython.display import display

from skimage import exposure
from scipy.signal import lfilter

import datetime

import warnings

import collections

## Functions for Loading Data and Building the Xarray
This project built on some existing work by Peter Tan. An output from Peter's urban change detection algorithm is raster files with all the relevant NBAR (analysis ready satellite derived surface reflectance readings) data saved to the output directory. To speed the loading and analysis during this script, this notebook will use those exisitng files if they are available. Otherwise it will load the data from the Digital Earth Australia archive.

### function: checkForLocalFiles

In [ ]:
def checkForLocalFiles(study_area):
    rootdir = os.listdir('../')
    if study_area in rootdir:
        return True
    else:
        return False

### function: getData

In [ ]:
def getData(study_area):
    # if the study area is a string, and is accessible locally, load it
    if isinstance(study_area, str):
        if checkForLocalFiles(study_area):
            data = getLocalData(study_area)
            return data
    # if the study area is a string and is on the list, load it
        else:
            data = DCLoadName(study_area)
    # if the study area is a list of coordinates, use them to load the data
    elif isinstance(study_area, list) and len(study_area) == 4:
        data = DCLoad(study_area)
        
    # if the study area isn't loaded locally, transfrom the DC originated xarray into "my" format  
    if not checkForLocalFiles(study_area):
        data = transformXarrayToCustomStyle(data)
        return data
    else:
        print('Data Loading Error')

### function: DCLoadName
This function is a wrapper for the DCLoad function, that allows previously used study areas to be easily restudied
by easily loading exactly the same area of interest (AOI).

In [ ]:
def DCLoadName(study_area):   
    if study_area == 'mtbarker':
        lat_min = -35.05
        lat_max = -35.08
        lon_min = 138.85
        lon_max = 138.895  
    elif study_area == 'swmelb':
        lat_min = -37.879
        lat_max = -37.91
        lon_min = 144.705
        lon_max = 144.76  
    elif study_area == 'gunghalin':
        lat_min = -35.18
        lat_max = -35.21
        lon_min = 149.14
        lon_max = 149.17
    elif study_area == 'goldengrove': 
        lat_min = -34.77
        lat_max = -34.8
        lon_min = 138.66
        lon_max = 138.73
    elif study_area == 'molonglo':
        lat_min = -35.3
        lat_max = -35.33
        lon_min = 149.015
        lon_max = 149.06
    elif study_area == 'nperth':
        lat_min = -31.686
        lat_max = -31.73
        lon_min = 115.79
        lon_max = 115.813
    elif study_area == 'swbris':
        lat_min = -27.66
        lat_max = -27.7 
        lon_min = 152.877
        lon_max = 152.93
    elif study_area == 'swsyd':
        lat_min = -33.993
        lat_max = -34.04
        lon_min = 150.715 
        lon_max = 150.78
    
    return DCLoad([lat_min, lat_max, lon_min, lon_max])

### function: DCLoad
This function is a variation of a datacube query supplied by Erin Telfer.

In [ ]:
def DCLoad(study_area):
    # to time how long the load takes
    start = datetime.datetime.now()
    print('Loading data') 
    print('Load Started At: ' + str(start))
    
    # define temporal range 
    start_of_epoch = '1987-01-01'
    end_of_epoch =  '2017-10-31'

    # define bands of interest
    bands_of_interest = ['blue', 'green', 'red', 
                         'nir', 'swir1', 'swir2']

    # Landsat sensors of interest are defined
    sensors = ['ls8', 'ls7', 'ls5'] 

    # unpack input parameter
    lat_min, lat_max, lon_min, lon_max = study_area    

    print('Bounding box: ' + str(lat_min) + ' S, ' + str(lon_min) +
          ' E to ' + str(lat_max) + ' S, ' + str(lon_max) + ' E' )
    print('Epoch: ' + start_of_epoch + ' to ' + end_of_epoch)
    print('Sensors: ' + str(sensors))
    print('Bands of Interest: ' + str(bands_of_interest))

    # create query
    query = {'time': (start_of_epoch, end_of_epoch),}
    query['x'] = (lon_min, lon_max)
    query['y'] = (lat_max, lat_min)
    query['crs'] = 'EPSG:4326'

    #Create cloud mask. This will define which pixel quality (PQ) artefacts are removed from the results.
    # It should be noted the "land_sea" code will remove all ocean/sea pixels.
    mask_components = {'cloud_acca':'no_cloud',
    'cloud_shadow_acca' :'no_cloud_shadow',
    'cloud_shadow_fmask' : 'no_cloud_shadow',
    'cloud_fmask' :'no_cloud',
    'blue_saturated' : False,
    'green_saturated' : False,
    'red_saturated' : False,
    'nir_saturated' : False,
    'swir1_saturated' : False,
    'swir2_saturated' : False,
    'contiguous':True,
    'land_sea': 'land'}

    # Connect to DataCube
    dc = datacube.Datacube(app='Urban Change Detection')
    
    # Data for each Landsat sensor is retrieved and saved in a dict for concatenation
    sensor_clean = {}
    
    for sensor in sensors:
        # Load the NBAR and corresponding PQ
        sensor_nbar = dc.load(product= sensor+'_nbar_albers', group_by='solar_day', 
                              measurements = bands_of_interest,  **query)
        sensor_pq = dc.load(product= sensor+'_pq_albers', group_by='solar_day', 
                            fuse_func=ga_pq_fuser, **query)

        # Retrieve the projection information before masking/sorting
        crs = sensor_nbar.crs
        crswkt = sensor_nbar.crs.wkt
        affine = sensor_nbar.affine        

        # Combing the pq so it is a single 
        sensor_all = xr.auto_combine([sensor_pq,sensor_nbar])
        sensor_clean[sensor] = sensor_all

        print('Loaded %s' % sensor) 

    print('Concatenating')
    nbar_clean = xr.concat(sensor_clean.values(), 'time')
    nbar_clean = nbar_clean.sortby('time')
    nbar_clean.attrs['crs'] = crs
    nbar_clean.attrs['affin|e'] = affine    

    print ('Load and Xarray build complete')
    print('Process took ' + str(datetime.datetime.now() - start))
    
    # return xarray
    return nbar_clean

### function: getLocalData

In [ ]:
def getLocalData(study_area):
    """A quick helper function to load the output files from Peter's code for the given location.
    It returns and Xarray of the landsat data for that study area."""
    # build a list of all files in the directory (ie the folder for that location)
    location = '../' + study_area + '/'
    files = os.listdir(location)

    print('Loading data from: ' + location)
    
    # build a list of all the NBAR*.img file names and which bands they represent
    NBARfiles = []
    bands = []
    for file in files:
        if file[-4::] == '.img' and file[0:4] == 'NBAR':
            NBARfiles.append(file)
            bands.append(file.split('NBAR_')[1].split('.img')[0])

    # open all the .img files with NBAR in the name, convert to numpy array, swap axes so order is (x, y, t)
    # and save to dict
    raw_data = {}
    for i in range(len(NBARfiles)):
        raw_data[bands[i]] = gdal.Open(location + NBARfiles[i]).ReadAsArray().swapaxes(0,2)
#     num_scenes = len(raw_data['red'][0][0])   # delete this?

    # build a list of all the dates represented by each band in the NBAR files
    # reuse the list of NBAR file names, but this time access the .hdr file
    in_dates = False
    dates = []
    for line in open(location + NBARfiles[0].split('.img')[0] + '.hdr'):
        if line[0] == '}':
            continue
        if in_dates:
            dates.append(line.split(',')[0].strip())
        if line[0:10] == 'band names':
            in_dates = True

    # save list of satellite originated bands
    sat_bands = bands.copy()

    # add the yet to be calculated derivative bands to the overall bands list
    bands += ['cloud_mask']

    # building the Xarray
    # define the size for the numpy array that will hold all the data for conversion into XArray
    x = len(raw_data['red'])
    y = len(raw_data['red'][0])
    t = len(raw_data['red'][0][0])
    n = len(bands)

    # create an empty numpy array of the correct size
    alldata = np.zeros((x, y, t, n), dtype=np.float32)

    # populate the numpy array with the satellite data
    # turn all no data NBAR values to NaNs
    for i in range(len(sat_bands)):
        alldata[:,:,:,i] = raw_data[sat_bands[i]]
        alldata[:,:,:,i][alldata[:,:,:,i] == -999] = np.nan

    # convert the numpy array into an xarray, with appropriate lables, and axes names
    data = xr.DataArray(alldata, coords = {'x':range(x), 'y':range(y), 'date':dates, 'band':bands},
                 dims=['x', 'y', 'date', 'band'])
    
    # import cloudmask and add to xarray
    cloudmask = gdal.Open(location + '/tsmask.img').ReadAsArray().swapaxes(0,2)
    data.loc[:,:,:,'cloud_mask'] = cloudmask
    
    return data

### function: transformXarrayToCustomStyle

In [7]:
def transformXarrayToCustomStyle(data_new):
    # loop through and extract all the values into a list of 3D numpy arrays
    bands = ['blue','green','red','nir','swir1','swir2','pixelquality']
    all_data = []
    for band in bands:
        all_data.append(data_new.variables[band].values.swapaxes(2,0).astype(np.float32))

    # replace 'pixelquality' with 'cloud_mask'  
    bands[bands.index('pixelquality')] = 'cloud_mask'

    # stack the list of 3D arrays along a 4th dimension
    data_flipped = np.stack(all_data, axis = -1)
    # set all bad NBAR values to np.nan
    data_flipped[data_flipped == -999] = np.nan
    # set all good pixels to 0 based on PQ indicator
    # see https://www.sciencedirect.com/science/article/pii/S0034425717301086 for PQ description
    data_flipped[data_flipped[:,:,:,-1] == 16383] = 0

    # save the new shape for use in defining the Xarray
    new_shape = data_flipped.shape

    # build a list of all the dates (as strings)
    dates = []
    for time in range(len(data_new.time)):
        dates.append(np.datetime_as_string(data_new.time[time].values)[0:10])

    # assemble the new Xarray
    newdatafixed = xr.DataArray(data_flipped, coords = {'x': range(new_shape[0]), 'y': range(new_shape[1]),
                                                        'date': dates, 'band': bands}, dims=['x','y','date','band'])
    return newdatafixed

# Training Data Generator Plot

## Functions

### function: drawTrainingPlot

In [ ]:
def drawTrainingPlot(study_area, scene_num, covertype, scene_picks_arr):
    """Allows easy extension to extra subplots in the training plot figure"""
    ax1, scene_picks_arr = drawTrainingScene(study_area, scene_num, covertype, scene_picks_arr)
    plt.draw()
    
    return scene_picks_arr

### function: drawTrainingScene

In [ ]:
def drawTrainingScene(study_area, scene_num, covertype, scene_picks_arr):
    """This function draws the desired scene (study area and scene number).
    It also presents the existing training data for that scene if any exists.
    It returns the axes object for the image, along with a numpy array which is
    the existing picks for that scene"""
    
    # get data for selected study area
#     data = getData(study_area)
    
    # colour map included incase of need to display false colour or other in the future
    # could change this to an ordereddict and remove the RGB list created below...?
    # or have RGB a list, and use that in data.sel(band=RGB).values
    colourmap = {'R':'red', 'G':'green', 'B':'blue'}
    
    # combine the data for the 3 bands to be displayed into a single numpy array
    h = data.shape[1]
    w = data.shape[0]
    t = data.shape[2]
    if scene_num > (t -1):
        scene_num = t - 1
    RGB = ['R','G','B']
    date = str(data[:,:,scene_num].date.values)
    
    # create array to store the RGB info in, and fill by looping through the colourmap variable
    # note the .T at the end, because the data array is setup as a (x,y,t), but imshow works (y,x)
    rawimg = np.zeros((h, w, 3), dtype=np.float32)
    for i in range(len(RGB)):     
        rawimg[:,:,i] = data[:,:,scene_num].sel(band=colourmap[RGB[i]]).T
        
    # equalizing for all bands together
    # goal is to make is human interpretable
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))    

    # displaying the results and formatting the axes etc
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('True Colour Landsat Scene, taken\n' + date + ', over ' + study_area)
    
   
    if scene_picks_arr is None:
        
        if study_area in trainingdata.index and scene_num in trainingdata.loc[study_area].index:
            # if there aren't any picks yet, make the array
            scene_picks_arr = np.zeros((h,w), dtype=np.float32)
            # fill it with np.nan
            scene_picks_arr[scene_picks_arr == 0] = np.nan
            # make a dict with the key (study_area, scene_num)
            scene_picks_arr = {(study_area, scene_num): scene_picks_arr}
            temp = trainingdata.loc[(study_area, scene_num)]
            # loop through all relevant training points, and populate the array
            for i in range(len(temp)):
                position = temp.iloc[i].name
                scene_picks_arr[(study_area, scene_num)][position[0], position[1]] = temp['landcover'].iloc[i]
        else:
            # if not the right scene/location combination, set to None
            scene_picks_arr = None
    
    # if there are picks, then plot them up, coloured as per the environment level variable colours
    # should I better tie cmap colours to colours
    if scene_picks_arr is not None and (study_area,scene_num) in (scene_picks_arr.keys()):
        cmap = colors.ListedColormap(colours)
        # plot the training pixels
        ax.imshow(scene_picks_arr[(study_area, scene_num)], cmap)
        legend_patches = []
        # build the legend
        for cover in landcover.keys():
            legend_patches.append(mpatches.Patch(color = colours[landcover[cover]-1], label = cover))
        ax.legend(handles = legend_patches)
    else:
        # if not the right scene/location combination, set to None
        scene_picks_arr = None
    
    return ax, scene_picks_arr

### function: train

In [ ]:
# some broad scope variables specific to the plotting that need setting up and seem to very fragile
# so I'm too scared to move them in case something breaks!
global xpos
global ypos
xpos = 0
ypos = 0
global scene_picks_arr
scene_picks_arr = None
colours = ['r', 'b', 'm', 'c']

def train(study_area, scene_num, covertype):

    def onclick(event):
        # defining what to do on a click event
        
        # I don't understand why this need to be declared global again, but it breaks without these lines
        global xpos
        global ypos
        # need to cast to int as result is a float, and can't index a list with a float
        xpos = int(event.xdata)
        ypos = int(event.ydata)
        # save the results of the click to the training data dataframe
        trainingdata.loc[(study_area, scene_num, ypos, xpos)] = landcover[covertype]
        # add the results to the current scenes overlay
        scene_picks_arr[(study_area,scene_num)][ypos, xpos] = landcover[covertype]
        # redraw with the trained pixels updated on the image
        drawTrainingPlot(study_area, scene_num, covertype, scene_picks_arr)
    
    # control the figure size
    fig = plt.figure(figsize=[10,10])
    axs = fig.axes
    plt.subplots_adjust(hspace = 0.6)
    
    # draw the figure
    global scene_picks_arr
    scene_picks_arr = drawTrainingPlot(study_area, scene_num, covertype, scene_picks_arr)
    #connect the click event action to the figure
    cid = fig.canvas.mpl_connect('button_press_event', onclick)

## Setting up broad scope variables

### Load Previous Training Data (or make blank dataframe)

In [8]:
# load previous training data
# by taking the last (ie most recent if the standard date is attached to the file) .pkl file
files = os.listdir('../')
pickles = []
for file in files:
    if file[-3::] == 'pkl':
        pickles.append(file)
trainingdata = pd.read_pickle('../' + pickles[-1])
# trainingdata = trainingdata.drop(columns = trainingdata.columns[1::])

# # setup a multilevel heirachrical index dataframe to store the results
# # storing the training data in this format is way more memory efficient than in an Xarray of same size as data
# # but it takes a lot of processing and manipulation to get it into a more useable form

# trainidx = pd.MultiIndex(levels = [[]]*4, labels = [[]]*4, names=['study_area', 'scene_num', 'row','column'])
# traincols = ['landcover']
# trainingdata = pd.DataFrame(index = trainidx, columns = traincols)

# view the current status
trainingdata

landcover    blue   green     red     nir  \
study_area scene_num row column                                              
mtbarker   1         38  105             1   247.0   434.0   344.0  4094.0   
                     33  104             1   420.0   717.0   560.0  3877.0   
                     32  105             1   420.0   677.0   524.0  3833.0   
                     28  106             1   362.0   717.0   524.0  3877.0   
                     26  102             1   324.0   636.0   452.0  4483.0   
                     24  100             1   305.0   555.0   380.0  4093.0   
                     25  109             1   324.0   596.0   416.0  4527.0   
                         116             1   363.0   677.0   524.0  3746.0   
                     29  148             1   439.0   958.0   739.0  5734.0   
                         150             1   362.0   757.0   560.0  5390.0   
                     31  152             1   458.0  1038.0   847.0  5691.0   
                     35  151             1   382.0   838.0   632.0  5820.0   
                     37  150             1   458.0   918.0   739.0  5906.0   
                     68  136             1   477.0   958.0   811.0  5305.0   
                     61  144             1   477.0   918.0   811.0  4874.0   
                     58  145             1   515.0  1038.0   883.0  5219.0   
                     55  147             1     NaN     NaN     NaN     NaN   
                     56  140             1   477.0   838.0   739.0  4571.0   
                     59  137             1   477.0   998.0   811.0  4917.0   
                     54  165             1   382.0   758.0   488.0  5563.0   
                     55  167             1   420.0   758.0   524.0  5606.0   
                     53  168             1   363.0   677.0   524.0  5348.0   
                     52  167             1   363.0   758.0   560.0  5262.0   
                     51  165             1   363.0   757.0   524.0  5348.0   
                     49  165             1   401.0   798.0   596.0  5348.0   
                     47  166             1   382.0   757.0   632.0  5391.0   
                     46  169             1   382.0   717.0   632.0  5262.0   
                     50  169             1   363.0   758.0   560.0  5606.0   
                     60  166             1   305.0   596.0   416.0  5434.0   
                     59  165             1   363.0   556.0   416.0  5348.0   
...                                    ...     ...     ...     ...     ...   
swsyd      830       117 10              1   170.0   347.0   362.0  2402.0   
                     124 7               1   170.0   347.0   303.0  2402.0   
                     105 56              1   170.0   421.0   245.0  2402.0   
                     97  56              1   133.0   347.0   303.0  2262.0   
                     210 18              1   243.0   713.0   654.0  3306.0   
                     204 21              1   316.0   713.0   712.0  2890.0   
                     192 27              1   352.0   858.0   770.0  3374.0   
                     191 25              1   424.0   858.0   770.0  3651.0   
                     238 21              1   496.0  1003.0  1061.0  3029.0   
                     239 33              1   352.0   858.0   829.0  3859.0   
                     233 41              1   243.0   494.0   479.0  2891.0   
                     201 262             1   281.0   641.0   363.0  4821.0   
                     194 258             1   281.0   714.0   363.0  5369.0   
                     189 253             1   317.0   713.0   479.0  5573.0   
                     175 250             1   317.0   713.0   362.0  6254.0   
                     234 177             2     NaN     NaN     NaN     NaN   
                     235 180             2     NaN     NaN     NaN     NaN   
                     230 129             2  1245.0  1722.0  1869.0  2334.0   
                     225 123             2  2386.0  313

### Other broad scope variables

In [21]:
# easier to work with integers than strings, so map the planned training classes to integers
landcover = {'vegetation':1,'urban':2,'earth':3,'water':4}
# range of pretermined study areas to use as sources for training data
study_areas = ['mtbarker', 'swmelb', 'gunghalin', 'goldengrove', 'molonglo', 'nperth', 'swbris', 'swsyd', 'custom']

# not in broad scope yet
sat_bands = ['blue','green','red','nir','swir1','swir2']
dc_bands = sat_bands.copy() + ['cloud_mask']

colours = ['r', 'b', 'm', 'c']

## The Training Figure

In [10]:
# create a study area drop down list and display it
study_area_dd = Dropdown(options=study_areas, value = study_areas[0], description='Study Area', disabled = False)
display(study_area_dd)

Dropdown(description='Study Area', options=('mtbarker', 'swmelb', 'gunghalin', 'goldengrove', 'molonglo', 'nperth', 'swbris', 'swsyd', 'custom'), value='mtbarker')

In [ ]:
# work with the value of the dropdown list
study_area = study_area_dd.value
if study_area == 'custom':
    coords = ['lat_min', 'lat_max', 'lon_min', 'lon_max']
    spatial_query = collections.OrderedDict()
    for coord in coords:
        spatial_query[coord] = input(coord + ': ')
    data = getData(list(spatial_query.values()))
else:
    data = getData(study_area)

print('\nStudy area data loaded.')

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    interact(train,
             study_area = fixed(study_area),
             scene_num = IntSlider(value = 1, min = 0, max = data.shape[2]-1,description = "Scene Number"),
             covertype = Dropdown(options=list(landcover.keys()), value=list(landcover.keys())[0], description='Landcover', disabled = False))

In [ ]:
# check the outputs of the training data generation process
trainingdata

# Training Results Manipulation and Classifier Training Function

## Building the training dataset

In [ ]:
# Aim is to get the data from the dataframe (which holds references to the pixel's location, along with the
# assigned class for that pixel), use it to extract the spectral data for that pixel, format it appropriately
# and pass it to the classification algorithm to teach it.

# useful variables for pulling out data from Xarray
sat_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
dc_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'cloud_mask']

# make the required columns
sat_bands_loc = []
for band in sat_bands:
    if band in trainingdata.columns:
        continue
    trainingdata[band] = np.nan
    sat_bands_loc.append(trainingdata.columns.get_loc(band))
    
# loop through the different locations used for the training data.
for loc in trainingdata.index.levels[0]:
    
    # build the Xarray for that location
    data = getData(loc)
    # only look at the training data for that location
    subset = trainingdata.loc[loc]
    # for each row (ie each pick) at that location
    for i in range(len(subset)):
        # unpack the multilevel pandas index into components for accessing the correct Xarray pixel
        scene, y, x = subset.iloc[i].name
        vals = data[x, y, scene].sel(band=dc_bands)
        if vals.sel(band='cloud_mask').values == 0:
            # if the pixel is valid (no cloud), take the spectral bands
            if np.isfinite(vals.sel(band=sat_bands).values).all():
                # if all the bands have readings (no NaNs), save the relevant bits into X and Y
                trainingdata.loc[(loc, scene, y, x), sat_bands] = vals.sel(band=sat_bands).values

# save the latest version of trainingdata somewhere good
time = str(datetime.datetime.now()).split('.')[0].replace(' ','_')
trainingdata.to_pickle('../traningdata_' + time + '.pkl')                
                
trainingdata

## function: makeClassifier

In [11]:
def makeClassifier(model):
    if model == 'svc':
        clf = svm.SVC()
    if model == 'rfc':
        clf = RandomForestClassifier()
    # scale and normalize the data
    X_fortraining = preprocessing.scale(trainingdata.dropna(axis=0, how = 'any')[sat_bands].values)
    X_fortraining = preprocessing.normalize(X_fortraining)

    # create a support vector classifer, and fit the data to it
    # might be worth trying a RandomForest Classifier
    clf.fit(X_fortraining,trainingdata.dropna(axis=0, how = 'any')['landcover'].values)
    return clf

# Formatting Remaining Data for Classification & Classifying It

In [12]:
def reformatAndClassify(study_area, clf):
    # get the data
    data = getData(study_area)
    
    #setting up the xarray to store the results for easy plotting later
    for newband in ['landcover','predicted_landcover']:
        temp = data[:,:,:].sel(band='red').copy()
        temp.band.values = newband
        temp.values[:] = np.nan
        data = xr.concat([data, temp], dim='band')

    # useful variable for down the track
    shape = data.values.shape

    start = datetime.datetime.now()
    print('Classifying ' + study_area + ' at ' + str(start))
    
    # classify one scene at a time, save the results to the xarray
    # rewrite this to work pixel by pixel on timeseries, will tie in better to change detection
    for scene in range(shape[2]):

        # setting up dataframe multilevel indexes
        col_idx = list(range(shape[0])) * shape[1]
        row_idx = []
        for i in range(shape[1]):
            row_idx += [i] * shape[0]
        scene_idx = [scene] * (shape[0] * shape[1])

        # reshape the data into a 2D flat array for scikit learn
        flattened = data[:,:,scene].sel(band=dc_bands).values.reshape(shape[0] * shape[1], len(dc_bands))

        # add the data to a new DataFrame, set up the columns and index
        alldata = pd.DataFrame(flattened)
        alldata.columns = dc_bands
        alldata['row'] = row_idx
        alldata['column'] = col_idx
        alldata['scene_num'] = scene_idx
        alldata['study_area'] = study_area
        alldata = alldata.set_index(['study_area','scene_num','row','column'])

        # join in the training data. This is a SQL left join, so only adds data to current study area
        alldata = alldata.reset_index().join(trainingdata[['landcover']], on=trainingdata.index.names).set_index(alldata.index.names)

        # reduce alldata down to valid pixels (ie cloudmask), and non-training pixels (ie landcover is still NaN)
        datatoclassify = alldata[alldata['cloud_mask'] == 0 & np.isnan(alldata['landcover'])].copy()
        # remove pixels with a np.nan as scikit-learn doesn't like them. Only keep spectral bands
        datatoclassify = datatoclassify[np.isnan(datatoclassify['landcover'])][sat_bands]
        # cast these relevant columns into a numpy array
        datatoclassify_np = np.array(datatoclassify)

        # to deal with an entirely clouded scene
        if len(datatoclassify_np) == 0:
            continue

        # scale and normalize the data so it resembles the training data.
        datatoclassify_np = preprocessing.scale(datatoclassify_np)
        datatoclassify_np = preprocessing.normalize(datatoclassify_np)

        # results of predict() are a 1 dimensional numpy array of the same length as the input data
        # assign these results to a new column in the dataframe
        datatoclassify['predicted_landcover'] = clf.predict(datatoclassify_np)

        # SQL left join the results back onto the original data
        alldata = alldata.reset_index().join(datatoclassify[['predicted_landcover']], on=trainingdata.index.names).set_index(alldata.index.names)

        #save the training data and classification results into the Xarray
        data[:,:,scene].loc[dict(band='landcover')] = alldata['landcover'].values.reshape(shape[0],shape[1])
        data[:,:,scene].loc[dict(band='predicted_landcover')] = alldata['predicted_landcover'].values.reshape(shape[0],shape[1])

    
    print('Time taken to classify ' + study_area + ' was ' + str(datetime.datetime.now() - start))
    return data

## function: drawClassifiedScene

# Viewing the Classification Results

In [ ]:
def drawClassifiedScene(data, scene_num, alpha):
    
    # colour map included incase of need to display false colour or other in the future
    # could change this to an ordereddict and remove the RGB list created below...?
    colourmap = {'R':'red', 'G':'green', 'B':'blue'}
    
    # combine the data for the 3 bands to be displayed into a single numpy array
    h = data.shape[1]
    w = data.shape[0]
    t = data.shape[2]
    
    if scene_num > (t -1):
        scene_num = t - 1
    RGB = ['R','G','B']
    date = str(data[:,:,scene_num].date.values)
    
    # create array to store the RGB info in, and fill by looping through the colourmap variable
    # note the .T at the end, because the data array is setup as a (x,y,t), but imshow works (y,x)
    rawimg = np.zeros((h, w, 3), dtype=np.float32)
    for i in range(len(RGB)):     
        rawimg[:,:,i] = data[:,:,scene_num].sel(band=colourmap[RGB[i]]).T
        
    # equalizing for all bands together
    # goal is to make is human interpretable
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))    

    # displaying the results and formatting the axes etc
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('True Colour Landsat Scene, taken\n' + date + ', over ' + study_area)
    
    # make the colour map for the cover classes
    cmap = colors.ListedColormap(colours)
    
    # draw the classification results and the training data results
    ax.imshow(data[:,:,scene_num].sel(band='predicted_landcover').values.T, cmap = cmap, alpha = alpha)
    ax.imshow(data[:,:,scene_num].sel(band='landcover').values.T, cmap = cmap, alpha = 1)
    
    #draw a legend for the classification colours
    legend_patches = []
    for cover in landcover.keys():
        legend_patches.append(mpatches.Patch(color = colours[landcover[cover]-1], label = cover))
    ax.legend(handles = legend_patches)
    
    return ax

## function: drawClassifiedPlots

In [ ]:
def drawClassifiedPlots(data, scene_num, alpha):
    
    ax1 = plt.subplot2grid([2,4],[0,0], rowspan = 2, colspan = 2)
    ax1.clear()
    ax1 = drawClassifiedScene(data, scene_num, alpha = 0)
    ax2 = plt.subplot2grid([2,4],[0,2], rowspan = 2, colspan = 2)
    ax2.clear()
    ax2 = drawClassifiedScene(data, scene_num, alpha)
    plt.draw()

## function: check

In [ ]:
def check(data, scene_num, alpha):
 
    # control the figure size
    fig = plt.figure(figsize=[12,7])
    axs = fig.axes
    plt.subplots_adjust(hspace = 0.6)
    
    # draw the figure
    drawClassifiedPlots(data, scene_num, alpha)

## Drawing the Reults

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    interact(check,
             data = fixed(data),
             scene_num = IntSlider(value = 1, min = 0, max = 2000,description = "Scene Number"),
             alpha= FloatSlider(value = 0.6, min = 0, max = 1, description = "Classification Transparency"))

# Time Series of Classifications into Change Detection

## function: modalFilter

In [13]:
# this function gets called a lot. Needs to be significantly sped up!
# dict instead of list.index()?
# remove first line
# remove length limit at start

def modalFilter(df, column, index, span = 10):
    df = df[~np.isnan(df[column])]
    if index > (len(df) - span) + 1:
        return np.nan
    else:
        mode_arr = df[column][index : index + span].mode().values
        if len(mode_arr) > 1:
            mode_arr2 = df[column][index : index + span + 1].mode().values
            if len(mode_arr2) > 1:
                heirarchy = [1, 3, 4, 2] # vegetation, earth, water, urban
                max_priority = 4
                i_max = 0
                for i in mode_arr:
                    if heirarchy.index(i) < max_priority:
                        max_priority = heirarchy.index(i)
                        i_max = i
                return i_max
            else:
                return mode_arr2[0] 
        else:
            return mode_arr[0]

## function: dateStringToFloat

In [14]:
def dateStringToFloat(s):
    year = int(s[0:4])
    month = int(s[5:7])
    
    if month in [1,2,3]:
        year += 0.125
    elif month in [4,5,6]:
        year += 0.375
    elif month in [7,8,9]:
        year += 0.625
    else:
        year == 0.875
    
    return year

## Running the Change Detection

In [23]:
def changeDetector(study_area, data, clftype):

    # setting up the results raster
    shape = data.shape
    changedates_arr = np.zeros((shape[1], shape[0]), dtype=np.float32)
    changedates_arr[changedates_arr == 0] = np.nan

    # variables for the modal filtering
    mode_span = 10
    MoM_span = 3

    start = datetime.datetime.now()
    print('Change detection for ' + study_area + ' start time: ' + str(start))

    # a very slow nested loop, keen to remove if possible
    for x in range(shape[0]):
        for y in range(shape[1]):
            
            # make a dataframe of the time-series of the predicted classifications
            pixeldata = pd.DataFrame(data[x, y, :].sel(band='predicted_landcover').values, index = data.date.values, columns = ['predicted_landcover'])
            # drop NaNs
            pixeldata = pixeldata.dropna(axis = 0, how='any')

            #make a new column to store mode in
            pixeldata['mode'] = np.nan
            pixeldata['mode_of_modes'] = np.nan
            mode_loc = pixeldata.columns.get_loc('mode')

            # remove any possibility of duplicate dates (based on an issue with swmelbourne study area)
            pixeldata = pixeldata[~pixeldata.index.duplicated(keep = 'first')]

            # description of current decision rule for assigning urban change:
            #  calculate the mode over each 10 scenes, and assign that value to the first scene of the 10
            #     if the first two modal values are both urban, assume already developed and move to next pixel
            #     if not, calculate the mode of the modes, over each 3 modes, assigning the value to the first
            #  find the first mode of modes that is urban for that pixel
            #     use that data (MOM_change date) to find the group of 3 modes that contributed to the mode of modes
            #     within the individual classifications for that pixel within that group of 30 classifications
            #        if there is an instance of 2 classifications in a row being urban, use the date of the first of those
            #        if not, simply use the first instance of an urban pixel in that range of 30

            # Ideas for improvement:
            #    - Deal with water pixels (eg if >25% of classifications are water, ignore)

            # Limitations of Method:
            #    - Many of them!
            #    - Algorithm needs first 20 to establish baseline - so it can't detect early change
            #    - Algorithm needs last 30 to build mode of modes - so it can't detect most recent change
            #    - It's very slow - mulitple nested loops. Main speed up would come from being able to do
            #       Modal filter as a df.apply(modalFilter)

            # find mode of each 10 scenes, and store at first index of range
            # for example the mode of scenes 10-19 will be stored in row 10.
            for row in range(0, len(pixeldata), mode_span):
                pixeldata.iloc[row, mode_loc] = modalFilter(pixeldata, 'predicted_landcover', row, span = mode_span)

            # if either of the first two modes are urban, assume pixel is already urban at start of landsat archive    
            if (pixeldata[~np.isnan(pixeldata['mode'])].iloc[0:2].values ==  2).any():
                continue

            # view or slice of data with modes
            modes = pixeldata[~np.isnan(pixeldata['mode'])]

            # third level of nested of loops :(
            # applying modal filter to the modes, to create mode of modes
            # save the result in the pixeldata dataframe
            for row in range(0,len(modes),MoM_span):
                pixeldata.loc[modes.iloc[row].name, 'mode_of_modes'] = modalFilter(modes, 'mode', row, span = MoM_span)

            # decision criteria
            if len(pixeldata[pixeldata['mode_of_modes'] == 2]) > 0:
                MoM_changedate = pixeldata[pixeldata['mode_of_modes'] == 2].iloc[0].name
                M_ss = pixeldata.loc[MoM_changedate::]
                M_changedate = M_ss[M_ss['mode'] == 2].iloc[0].name
                M_changedate_loc = pixeldata.index.get_loc(M_changedate)
                pix_ss = pixeldata.iloc[M_changedate_loc - mode_span : M_changedate_loc + mode_span]
                twoinarow = pix_ss[(pix_ss['predicted_landcover'] == pix_ss['predicted_landcover'].shift(-1)) & 
                                    (pix_ss['predicted_landcover'][pix_ss['predicted_landcover'] == 2])]
                if len(twoinarow) > 0:
                    changedate = twoinarow.iloc[0].name
                    changedates_arr[y, x] = dateStringToFloat(changedate)
                else:
                    changedate = pix_ss[pix_ss['predicted_landcover'] == 2].iloc[0].name
                changedates_arr[y, x] = dateStringToFloat(changedate)

    # print how long the modal filtering and change detection took                
    print(study_area + ' processing time: ' + str(datetime.datetime.now() - start))           

    # save the results to a .pkl for future access
    results_save_location = '../' + study_area + '/changeresults_' + clftype + '.pkl'
    changedates_arr.dump(results_save_location)  
    print('Results have been saved to', results_save_location, '\n')

## Looping Through All Study Areas

In [22]:
bigrunstart = datetime.datetime.now()
print('All Study Areas Loop Commenced at: ' + str(bigrunstart) + '\n')

for clftype in ['rfc','svc']:
    for study_area in  ['mtbarker','swmelb','gunghalin']:    #study_areas[0:-1]:
        clf = makeClassifier(clftype)  #valid options are 'rfc' and 'svc'
        classified_data = reformatAndClassify(study_area, clf)
        changeDetector(study_area, classified_data, clftype)
    
print('\nAll study areas loop finished at: ' + str(datetime.datetime.now()))

All Study Areas Loop Commenced at: 2018-01-16 16:59:16.606058

Loading data from: ../mtbarker/
Classifying mtbarker at 2018-01-16 16:59:24.896896


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Time taken to classify mtbarker was 0:01:59.416662
Change detection for mtbarker start time: 2018-01-16 17:01:24.323128
mtbarker processing time: 1:11:14.320063
Results have been saved to ../mtbarker/changeresults_svm_overnight.pkl 

Loading data from: ../swmelb/
Classifying swmelb at 2018-01-16 18:13:08.077845


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Time taken to classify swmelb was 0:03:58.867840
Change detection for swmelb start time: 2018-01-16 18:17:06.950562
swmelb processing time: 2:37:19.101395
Results have been saved to ../swmelb/changeresults_svm_overnight.pkl 

Loading data from: ../gunghalin/
Classifying gunghalin at 2018-01-16 20:55:14.489819


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Time taken to classify gunghalin was 0:03:32.667342
Change detection for gunghalin start time: 2018-01-16 20:58:47.171499
gunghalin processing time: 1:11:34.168150
Results have been saved to ../gunghalin/changeresults_svm_overnight.pkl 

Loading data from: ../mtbarker/
Classifying mtbarker at 2018-01-16 22:10:27.981889
Time taken to classify mtbarker was 0:05:51.128539
Change detection for mtbarker start time: 2018-01-16 22:16:19.179498
mtbarker processing time: 1:10:33.965798
Results have been saved to ../mtbarker/changeresults_svm_overnight.pkl 

Loading data from: ../swmelb/
Classifying swmelb at 2018-01-16 23:27:20.011729
Time taken to classify swmelb was 0:12:35.162482
Change detection for swmelb start time: 2018-01-16 23:39:55.201492
swmelb processing time: 2:41:20.920893
Results have been saved to ../swmelb/changeresults_svm_overnight.pkl 

Loading data from: ../gunghalin/
Classifying gunghalin at 2018-01-17 02:21:28.580838
Time taken to classify gunghalin was 0:08:56.095145
Cha

## Modal Filter Testing and Example

In [ ]:
# mt barker pixel that definitely changed
# pixeldata = pd.DataFrame(data[129, 39, :].sel(band='predicted_landcover').values, index = data.date.values, columns = ['predicted_landcover'])

# mt barker pixel that was always urban
# pixeldata = pd.DataFrame(data[40, 63, :].sel(band='predicted_landcover').values, index = data.date.values, columns = ['predicted_landcover'])

# mt barker pixel that was exception causing
pixeldata = pd.DataFrame(data[106, 120, :].sel(band='predicted_landcover').values, index = data.date.values, columns = ['predicted_landcover'])

pixeldata = pixeldata.dropna(axis = 0, how='any')
pixeldata['mode'] = np.nan
pixeldata['mode_of_modes'] = np.nan
mode_loc = pixeldata.columns.get_loc('mode')


for row in range(0, len(pixeldata), mode_span):
    pixeldata.iloc[row, mode_loc] = modalFilter(pixeldata, 'predicted_landcover', row, span = mode_span)

# # if either of the first two modes are urban, assume pixel is already urban at start of landsat archive    
if (pixeldata[~np.isnan(pixeldata['mode'])].iloc[0:2].values ==  2).any():
    print('Already Urban')

modes = pixeldata[~np.isnan(pixeldata['mode'])]

for row in range(0,len(modes),MoM_span):
    pixeldata.loc[modes.iloc[row].name, 'mode_of_modes'] = modalFilter(modes, 'mode', row, span = MoM_span)
    
if len(pixeldata[pixeldata['mode_of_modes'] == 2]) > 0:
    MoM_changedate = pixeldata[pixeldata['mode_of_modes'] == 2].iloc[0].name
    M_ss = pixeldata.loc[MoM_changedate::]
    M_changedate = M_ss[M_ss['mode'] == 2].iloc[0].name
    M_changedate_loc = pixeldata.index.get_loc(M_changedate)
    
    pix_ss = pixeldata.iloc[M_changedate_loc - mode_span : M_changedate_loc + mode_span]
# pixeldata[np.isfinite(pixeldata['mode_of_modes'])]
    twoinarow = pix_ss[(pix_ss['predicted_landcover'] == pix_ss['predicted_landcover'].shift(-1)) & 
                        (pix_ss['predicted_landcover'][pix_ss['predicted_landcover'] == 2])]

    if len(twoinarow) > 0:
        changedate = twoinarow.iloc[0].name
    else:
        changedate = pix_ss[pix_ss['predicted_landcover'] == 2].iloc[0].name
changedate

# Drawing the Change Detection Results

## drawAnalysedScene()

In [ ]:
def drawAnalysedScene(data, scene_num, alpha, change_grid):
    
    # colour map included incase of need to display false colour or other in the future
    # could change this to an ordereddict and remove the RGB list created below...?
    colourmap = {'R':'red', 'G':'green', 'B':'blue'}
    
    # combine the data for the 3 bands to be displayed into a single numpy array
    h = data.shape[1]
    w = data.shape[0]
    t = data.shape[2]
    
    if scene_num > (t -1):
        scene_num = t - 1
    RGB = ['R','G','B']
    date = str(data[:,:,scene_num].date.values)
    
    # create array to store the RGB info in, and fill by looping through the colourmap variable
    # note the .T at the end, because the data array is setup as a (x,y,t), but imshow works (y,x)
    rawimg = np.zeros((h, w, 3), dtype=np.float32)
    for i in range(len(RGB)):     
        rawimg[:,:,i] = data[:,:,scene_num].sel(band=colourmap[RGB[i]]).T
        
    # equalizing for all bands together
    # goal is to make is human interpretable
    img_toshow = exposure.equalize_hist(rawimg, mask = np.isfinite(rawimg))    

    # displaying the results and formatting the axes etc
    plt.imshow(img_toshow)
    ax = plt.gca()
    ax.set_title('True Colour Landsat Scene, taken\n' + date + ', over ' + study_area)
    
    # define the current colour map to display the change results raster properly
    current_cmap = matplotlib.cm.get_cmap('Reds_r')
    current_cmap.set_under('k', alpha=0.0)
    current_cmap.set_over('r', alpha=1.0)
    current_cmap.set_bad('k', alpha=0.0)  
    
    # draw the change detection results mask
    ax.imshow(change_grid, alpha = alpha, interpolation='none', cmap = current_cmap, clim = [0.5, 0.6])
    return ax

## drawAnalysedPlots()

In [ ]:
def drawAnalysedPlots(data, left_scene_num, left_alpha, left_change_grid,
                      right_scene_num, right_alpha, right_change_grid):
    
    ax1 = plt.subplot2grid([2,4],[0,0], rowspan = 2, colspan = 2)
    ax1.clear()
    ax1 = drawAnalysedScene(data, left_scene_num, left_alpha, left_change_grid)
    ax2 = plt.subplot2grid([2,4],[0,2], rowspan = 2, colspan = 2)
    ax2.clear()
    ax2 = drawAnalysedScene(data, right_scene_num, right_alpha, right_change_grid)
    plt.draw()

## function: results

In [ ]:
def results(data, left_scene_num, left_alpha, left_change_grid,
                      right_scene_num, right_alpha, right_change_grid):
 
    # control the figure size
    fig = plt.figure(figsize=[12,7])
#     axs = fig.axes
#     plt.subplots_adjust(hspace = 0.6)
    
    # draw the figure
    drawAnalysedPlots(data, left_scene_num, left_alpha, left_change_grid,
                      right_scene_num, right_alpha, right_change_grid)

## Open Existing Change Results

In [ ]:
# get the results ready for comparison
# no change should be np.NaN
# chanage = 1
change = gdal.Open('../' + study_area + '/change_time.img').ReadAsArray()
change[change == 0] = np.nan

peterchange = change.copy()
peterchange[np.isfinite(peterchange)] = 1

# mikechange = changedates_arr.copy()
# mikechange[np.isfinite(mikechange)] = 1
results_save_location = '../' + study_area + '/changeresults_svm2.pkl'
mike_results = np.load(results_save_location)
mikechange = mike_results.copy()
mikechange[np.isfinite(mikechange)] = 1

num_scenes = data.shape[2]

## Draw Results Analysis Plots

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    interact(results,
             data = fixed(data),
             left_scene_num = IntSlider(value = 1, min = 0, max = num_scenes -1 ,description = "Scene Number"),
             left_alpha= FloatSlider(value = 0.6, min = 0, max = 1, description = "Left Alpha"),
             left_change_grid = fixed(peterchange),
             right_scene_num = IntSlider(value = num_scenes - 1, min = 0, max = num_scenes - 1,description = "Scene Number"),
             right_alpha= FloatSlider(value = 0.6, min = 0, max = 1, description = "Right Alpha"),
             right_change_grid = fixed(mikechange))

In [ ]:
% matplotlib inline
print(np.nanmean(mike_results - change))
imshow(mike_results - change)
plt.colorbar()

In [ ]:
change